## Setup

Mount google drive if on colab:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Download modules:

In [8]:
!pip install langchain langchain_huggingface langdetect

ERROR: Could not find a version that satisfies the requirement langchain.chat_models.huggingface (from versions: none)
ERROR: No matching distribution found for langchain.chat_models.huggingface


Imports:

In [15]:
# Imports

from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain.prompts import ChatPromptTemplate
from langchain_core.tools import tool, Tool
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor

from transformers import pipeline
from langchain.tools import Tool

from langdetect import detect, DetectorFactory
import langdetect.lang_detect_exception

from transformers import AutoTokenizer, MT5ForConditionalGeneration

import torch
import pandas as pd
import json

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [2]:
# Use GPU

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

We load the data below in the pipeline implementation.

Models:

We use the UBC Toucan model for translation:

To translate using Toucan models, we use the target language ISO-3 code as preix.

In [3]:
lang_names={
    "aar": "Afar",
    "ach": "Acholi",
    "afr": "Afrikaans",
    "aka": "Akan",
    "amh": "Amharic",
    "bam": "Bambara",
    "bas": "Basaa",
    "bem": "Bemba",
    "btg": "Bete Gagnoa",
    "eng": "English",
    "ewe": "Ewe",
    "fon": "Fon",
    "fra": "French",
    "hau": "Hausa",
    "ibo": "Igbo",
    "kbp": "Kabiye",
    "lgg": "Lugbara",
    "lug": "Luganda",
    "mlg": "Malagasy",
    "nyn": "Nyakore",
    "orm": "Oromo",
    "som": "Somali",
    "sot": "Sesotho",
    "swa": "Swahili",
    "tir": "Tigrinya",
    "yor": "Yoruba",
    "teo": "Ateso",
    "gez": "Geez",
    "wal": "Wolaytta",
    "fan": "Fang",
    "kau": "Kanuri",
    "kin": "Kinyawanda",
    "kon": "Kongo",
    "lin": "Lingala",
    "nya": "Chichewa",
    "pcm": "Nigerian Pidgin",
    "ssw": "Siswati",
    "tsn": "Setswana",
    "tso": "Tsonga",
    "twi": "Twi",
    "wol": "Wolof",
    "xho": "Xhosa",
    "zul": "Zulu",
    "nnb": "Nande",
    "swc": "Swahili Congo",
    "ara": "Arabic"
}


In [4]:
# Initialize our Toucan model for African language translation
toucan_tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/toucan-base")
toucan_model = MT5ForConditionalGeneration.from_pretrained("UBC-NLP/toucan-base", torch_dtype=torch.float16, device_map="auto")
toucan_model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

For the rest of the pipeline, we use the IBM Granite model:

In [5]:
granite_model = HuggingFacePipeline.from_model_id(
    model_id="ibm-granite/granite-3.2-2b-instruct",
    task="text-generation",
    pipeline_kwargs={
        "return_full_text": False,
        "do_sample": False,
        "max_new_tokens": 300
    }
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


NameError: name 'kwargs' is not defined

In [6]:
chat = ChatHuggingFace(llm=granite_model, temperature=.7)

## Language Detection & Translation

In [7]:
DetectorFactory.seed = 0

langdetect_to_toucan = {
    "en": "eng", "fr": "fra", "sw": "swa", "yo": "yor", "ha": "hau",
    "ig": "ibo", "ar": "ara", "xh": "xho", "zu": "zul", "am": "amh",
    "so": "som", "st": "sot", "rw": "kin", "lg": "lug", "ln": "lin",
    "sn": "tsn", "ss": "ssw", "ny": "nya", "mg": "mlg", "om": "orm",
    "ti": "tir", "nso": "tsn", "tn": "tsn", "ts": "tso", "tw": "twi",
    "wo": "wol", "kg": "kon", "ee": "ewe", "ff": "fan", "pcm": "pcm",
    "arq": "ara", "ffm": "fan", "kab": "kab", "bm": "bam", "nyn": "nyn"
}

def detect_language(text):
    try:
        detected_lang = detect(text)
        return langdetect_to_toucan.get(detected_lang, "unknown")
    except langdetect.lang_detect_exception.LangDetectException:
        return "unknown"

def detect_and_append_language(df, text_column="sentence"):
    df["detected_language"] = df[text_column].apply(detect_language)
    return df

# The following functions are to be used for tool creation, to bind to our main llm model.

def translate_if_needed(text):
  """Detect whether the text is in English, and translate it to English using Toucan if it is not."""
  detected_lang = detect_language(text)
  if detected_lang != "eng":
      # ranslation step if the language is not English
      # Add 'eng: ' to start of string so that Toucan knows to translate to English
      text = "eng: " + text
      # Call
      input_ids = toucan_tokenizer(text, return_tensors="pt", max_length=1024, truncation=True).to("cuda:0")
      with torch.no_grad():
        generated_ids = toucan_model.generate(**input_ids, num_beams=5, max_new_tokens=len(text), do_sample=True, temperature=0.6, top_p=0.9)
      english_text = toucan_tokenizer.batch_decode(generated_ids, skip_special_tokens=True,  skip_prompt=True)[0]
  # If detected language is English we just keep it as is
  else:
      english_text = text
  return english_text

def analyze_sentiment(review: str) -> str:
    sentiment_pipeline = pipeline("sentiment-analysis", model="tabularisai/multilingual-sentiment-analysis")
    result = sentiment_pipeline(review)
    if result[0]['label'] == 'Neutral':
        return "mixed"
    if result[0]['label'] == 'Negative':
        return "negative"
    if result[0]['label'] == 'Positive':
        return "positive"

def analyze_toxicity(text: str) -> str:
    toxicity_pipeline = pipeline("text-classification", model="unitary/toxic-bert")
    result = toxicity_pipeline(text)
    if result[0]['score'] > 0.5:
        return "toxic"
    else:
        return "non-toxic"

In [8]:
# Tool 1 of 3: Translation

translation_tool = Tool(
    name="African Language to English Translation Tool",
    description="Detects whether a language is African and translates it to English if so.",
    func=translate_if_needed,
    return_direct=True
)

# Test out the translation tool directly
if __name__ == "__main__":
    example_sentence = "Nilifikiri itakuwa mbovu, kumbe app hii ya mazoezi inanifanya nihisi nipo gym halisi, nimeipenda!"
    result = translation_tool.func(example_sentence)
    print(f"{result}")

I thought it would be bad, but this exercise app makes me feel I'm a real gym, thank you!


In [9]:
# Tool 2 of 3: Sentiment Classifier
sentiment_tool = Tool(
    name="Sentiment Analysis Tool",
    description="Analyzes the sentiment of a given text and returns the sentiment label. Input should be a string containing the text to analyze.",
    func=analyze_sentiment,
    return_direct=True
)

# Test the tool directly
if __name__ == "__main__":
  example_sentence = "This product was pretty good!"
  result = sentiment_tool.func(example_sentence)
  print(f"{result}")

Device set to use cuda:0


positive


In [10]:
# Tool 3 of 3: Toxicity classifier
toxicity_tool = Tool(
    name="Toxicity Analysis Tool",
    description="Analyzes the toxicity of a given text and returns the toxicity label. Input should be a string containing the text to analyze.",
    func=analyze_toxicity,
    return_direct=True
)

# Test the tool directly
if __name__ == "__main__":
  example_sentence = ""
  result = toxicity_tool.func(example_sentence)
  print(f"{result}")

Device set to use cuda:0


non-toxic


In [93]:
# The following prompt simulates an agentic workflow; Granite uses tools built from other LLMs to complete each task.

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert in sentiment and toxicity analysis. The user will provide you a query in the form "Task: sentence" below.
    If the sentence is not in English, translate it to English using the translation_tool.
    If the task is "Please perform toxicity analysis:" or similar, use the toxicity_tool to get a toxicity label for the sentence.
    If the task is "Please perform sentiment analysis:" or similar, use the sentiment_tool to get a sentiment label for the sentence.
    Explain your reasoning in 25 words or less for the sentiment or toxicity label, and include the explanation in your response.
    Finally, if the task is "Please perform detoxification:" or similar, return a detoxified version of the sentence with toxicity removed if it is toxic. If it is non-toxic, simply repeat the original sentence.
    When you return a sentence this way, DO NOT INCLUDE "Please perform detoxification:" in your response.
    Do not include any additional text in your output, only the JSON object.

    {format_instructions}

    Sentence: "{input}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

format_instructions = """
Return only a valid JSON object with no additional text or formatting.
For toxicity analysis, the JSON object must contain exactly the following keys:
{
  "toxicity": "toxic" | "non-toxic"
  "toxicity_explanation": "A brief reason for this classification in 25 words or less."
}
For sentiment analysis, the JSON object must contain exactly the following keys:
{
  "sentiment": "positive" | "mixed" | "negative",
  "sentiment_explanation": "A brief reason for this classification in 25 words or less."
}
For detoxification, the JSON object must contain exactly the following keys:
{
  "detoxified_text": "<The original sentence if it is non-toxic; DO NOT INCLUDE "Please perform detoxification:"> | <A re-written version of the original sentence with problematic language removed, if it is toxic ; DO NOT INCLUDE "Please perform detoxification:">"
}
"""

In [94]:
# Initialize our agent with the tools we made earlier, allowing it to call on multiple other LLMs for each task

tools = [sentiment_tool, translation_tool, toxicity_tool]

# We use a tool calling agent to take advantage of our other LLM tools:

agent = create_tool_calling_agent(
    tools = tools,
    llm=chat,
    prompt=prompt
)

In [95]:
# Initialize our agent_executor and test sentiment analysis

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
agent_executor.invoke({"input": "Please perform sentiment analysis: This product is okay, but I've seen better.", "format_instructions": format_instructions})['output']

'{\n  "sentiment": "mixed",\n  "sentiment_explanation": "The sentence expresses a mixed sentiment. \'okay\' indicates a neutral to positive view, while \'I\'ve seen better\' implies a comparison to superior alternatives."\n}'

In [97]:
# Test out toxicity analysis

agent_executor.invoke({"input": "Please perform toxicity analysis: You're a stupid idiot and nobody cares about you.", "format_instructions": format_instructions})['output']

'{\n  "toxicity": "toxic",\n  "toxicity_explanation": "The sentence contains direct personal attacks and disrespectful language, indicating high toxicity."\n}'

In [98]:
# Test out detoxification

agent_executor.invoke({"input": "Please perform detoxification: You're a stupid idiot and nobody cares about you.", "format_instructions": format_instructions})['output']

'{\n  "detoxified_text": "There\'s no need to resort to such language. Everyone deserves respect."\n}'

In [99]:
# Test out detoxification with a non-toxic sentence (should just return the original or similar)

agent_executor.invoke({"input": "Please perform toxicity analysis: Hi! How's it going?", "format_instructions": format_instructions})['output']

'{\n  "toxicity": "non-toxic",\n  "toxicity_explanation": "The sentence is a polite greeting and does not contain any toxic language."\n}'

In [100]:
# Test out multilingual sentiment analysis

agent_executor.invoke({"input": "Please perform sentiment analysis: Kiukweli, huu mchezo mpya una burudani, lakini huwezi kusonga mbele bila kununua vipengele vya ziada, inaboa!", "format_instructions": format_instructions})['output']

'{\n  "sentiment": "negative",\n  "sentiment_explanation": "The sentiment is negative due to the use of critical language (\'mchezo mpya una burudani\', \'huwezi kusonga mbele bila kununua vipengele vya ziada\') and the exclamation \'inaboa\', indicating frustration or disappointment."\n}'

## Running the data:

In [51]:
sentiment_test_path = "/content/drive/MyDrive/MDS/COLX_565/data/Milstone-2-multilingual-sentiment-test-solutions.csv"
m2_multilang_sentiment_df = pd.read_csv(sentiment_test_path)

m2_multilang_sentiment_df.head()

,sentence id,sentence,class-label
0,1,Ni kipaji gani! Hiki kipikiosafi kinafanya kaz...,mixed
1,2,"Ninapenda ubora wa picha, ila simu hii inajifu...",mixed
2,3,"Kweli, kozi hii ya mtandaoni inavutia, lakini ...",negative
3,4,"Nashukuru huduma ya haraka, lakini chakula kim...",negative
4,5,"Haya magari mapya ni mazuri, ila bei yake 'ina...",mixed


In [75]:
# Function to invoke our model on the sentiment data

def process_sentiment_row(text, max_retries=3):
  """Invokes our agentic model with a prompt requesting sentiment analysis on the input text."""
    for attempt in range(max_retries):
        try:
            output = json.loads(agent_executor.invoke({"input": f"Please perform sentiment analysis: {text}", "format_instructions": format_instructions})['output'])

            # Get the sentiment label and explanation
            sentiment_label = output['sentiment']
            sentiment_explanation = output['sentiment_explanation']

            # Ensure both keys are present, else raise an error and re-prompt
            if sentiment_label is None or sentiment_explanation is None:
                raise ValueError("Missing sentiment_label or sentiment_explanation in output.")

            return sentiment_label, sentiment_explanation

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for text: {text[:30]}... Error: {e}")

    # If all attempts fail, return default values
    return "Failed to process after 3 attempts."

In [107]:
# Function to invoke our model on the toxicity data

def process_toxicity_row(text, max_retries=3):
  """Invokes our agentic model with a prompt requesting sentiment analysis on the input text."""
  for attempt in range(max_retries):
        try:
            output = json.loads(agent_executor.invoke({"input": f"Please perform toxicity analysis: {text}", "format_instructions": format_instructions})['output'])
            detox_output = json.loads(agent_executor.invoke({"input": f"Please perform detoxification: {text}", "format_instructions": format_instructions})['output'])

            # Get the toxicity label and explanation + detoxified text
            toxicity_label = output['toxicity']
            toxicity_explanation = output['toxicity_explanation']
            detoxified_text = detox_output['detoxified_text']

            # Ensure all keys are present, else raise an error and re-prompt
            if toxicity_label is None or toxicity_explanation is None or detoxified_text is None:
                raise ValueError("Missing toxicity_label, toxicity_explanation, or detoxified_text in output.")

            return toxicity_label, toxicity_explanation, detoxified_text

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for text: {text[:30]}... Error: {e}")

    # If all attempts fail, return default values
  return "Failed to process after 3 attempts."

In [108]:
process_toxicity_row("You're a stupid idiot and nobody cares about you.")

('toxic',
 'The sentence contains direct personal attacks and disrespectful language, indicating high toxicity.',
 "There's no need to resort to such language. Everyone deserves respect.")

In [82]:
# Loop through our sentiment df and invoke the agent on each example

sentiment_results = []
for sentence in m2_multilang_sentiment_df["sentence"]:
    sentiment, explanation = process_sentiment_row(sentence)
    sentiment_results.append({"sentiment": sentiment, "explanation": explanation})

m2_multilang_sentiment_df["prediction"] = [res["sentiment"] for res in sentiment_results]
m2_multilang_sentiment_df["explanation"] = [res["explanation"] for res in sentiment_results]

true_sentiment = m2_multilang_sentiment_df["class-label"]
pred_sentiment = m2_multilang_sentiment_df["prediction"]

precision, recall, f1, _ = precision_recall_fscore_support(true_sentiment, pred_sentiment, average="macro")
accuracy = accuracy_score(true_sentiment, pred_sentiment)

In [101]:
# Load toxicity data

toxicity_test_path = "/content/drive/MyDrive/MDS/COLX_565/data/Milestone-2-toxic-test-solutions.csv"
m2_toxicity_df = pd.read_csv(toxicity_test_path)

m2_toxicity_df.head()

,data_id,sample_id,text,source_label,platform
0,111252,0,I know he's an idiot I think he might of been ...,toxic,wikipedia
1,152758,1,==my butthole== i love buttholes. they tast...,toxic,wikipedia
2,126091,2,This is absolute bullshit! We fucking knew thi...,non-toxic,fox news
3,28115,3,Moochelle Nobama takes swipe at Trump it sez. ...,toxic,fox news
4,2997,4,"2. \t> Lol, I guessed IPN (the Institue of Nat...",toxic,reddit


In [110]:
# Loop through the toxicity data and invoke the agent on each example

toxicity_results = []
for sentence in m2_toxicity_df["text"]:
    toxicity, explanation, detoxified_text = process_toxicity_row(sentence)
    toxicity_results.append({"toxicity": toxicity, "explanation": explanation, "detoxified_text": detoxified_text})

m2_toxicity_df["prediction"] = [res["toxicity"] for res in toxicity_results]
m2_toxicity_df["explanation"] = [res["explanation"] for res in toxicity_results]
m2_toxicity_df["detoxified_text"] = [res["detoxified_text"] for res in toxicity_results]

# compute evaluation metrics
true_toxicity = m2_toxicity_df["source_label"]
pred_toxicity = m2_toxicity_df["prediction"]

tox_precision, tox_recall, tox_f1, _ = precision_recall_fscore_support(true_toxicity, pred_toxicity, average="macro")
tox_accuracy = accuracy_score(true_toxicity, pred_toxicity)

In [113]:
m2_multilang_sentiment_df.to_csv("sentiment_analysis_results.csv", index=False)
m2_toxicity_df.to_csv("toxicity_analysis_results.csv", index=False)

In [84]:
# Let's see how our agent did:

print("Sentiment Analysis Evaluation:")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

Sentiment Analysis Evaluation:
Accuracy: 0.650
Precision: 0.635
Recall: 0.664
F1 Score: 0.634


In [111]:
print("\nToxicity Detection Evaluation:")
print(f"Accuracy: {tox_accuracy:.3f} ")
print(f"Precision: {tox_precision:.3f}")
print(f"Recall: {tox_recall:.3f}")
print(f"F1 Score: {tox_f1:.3f}")


Toxicity Detection Evaluation:
Accuracy: 0.560 
Precision: 0.292
Recall: 0.467
F1 Score: 0.359


Detoxify:

In [112]:
# Display first few rows of the DataFrame
print(m2_toxicity_df[["text", "prediction", "detoxified_text"]].head())

                                                text prediction  \
0  I know he's an idiot I think he might of been ...      toxic   
1    ==my butthole==  i love buttholes. they tast...      toxic   
2  This is absolute bullshit! We fucking knew thi...      toxic   
3  Moochelle Nobama takes swipe at Trump it sez. ...      toxic   
4  2. \t> Lol, I guessed IPN (the Institue of Nat...      toxic   

                                     detoxified_text  
0  I have an opinion that he might have been held...  
1  I love buttholes. They taste good. I will cont...  
2  This situation is concerning. We anticipated t...  
3  Michelle Obama takes a swipe at Trump, he shou...  
4  I anticipated IPN (the Institute of National R...  
